In [2]:
import pandas as pd

In [ ]:
def create_table_query(df, table_name):
    query = [f"CREATE TABLE {table_name} ("]

    for i, row in df.iterrows():
        column_name = row["필드.1"]
        data_type = row["타입.1"]
        length = row["길이.1"]
        description = row["설명.1"]

        query.append(
            f"    {column_name} {data_type}({length}) {f'COMMENT "{description}"' if row['NotNull.1'] is not None else ''},"
        )

    query[-1] = query[-1][:-1]
    query.append(");")

    return query


def insert_date_query(df, table_name):
    query = ["SELECT"]

    for i, col in enumerate(df["필드.1"]):
        if i == 0:
            query.append(f"    {col}")
            continue

        query.append(f"    , {col}")

    query.append("FROM")
    query.append(f"    {table_name}")

    query.append("WHERE")
    query.append("    1=1")
    query.append("    AND INSERT_DT >= TO_DATE(:start_date, 'YYYY-MM-DD HH24:MI:SS')")
    query.append("    AND INSERT_DT <  TO_DATE(:end_date,   'YYYY-MM-DD HH24:MI:SS')")

    return query


def update_date_query(df, table_name):
    query = ["SELECT"]

    for i, col in enumerate(df["필드.1"]):
        if i == 0:
            query.append(f"    {col}")
            continue

        query.append(f"    , {col}")

    query.append("FROM")
    query.append(f"    {table_name}")

    query.append("WHERE")
    query.append("    1=1")
    query.append("    AND UPDATE_DT >= TO_DATE(:start_date, 'YYYY-MM-DD HH24:MI:SS')")
    query.append("    AND UPDATE_DT <  TO_DATE(:end_date,   'YYYY-MM-DD HH24:MI:SS')")

    return query


def ods_extract_query(df, table_name):
    query = []
    query.extend(insert_date_query(df, table_name))
    query.extend(["UNION"])
    query.extend(update_date_query(df, table_name))
    query.extend(";")

    return query


def write_query(query, table_name):
    with open(f"data/output/queries/{table_name}.sql", "w") as f:
        for line in query:
            f.write(line)
            f.write("\n")


def run(df, table_name):
    query = ods_extract_query(df, table_name)
    write_query(query, table_name)


In [ ]:
excel_table_path = "./data/input/tables/table.xlsx"

tables = pd.read_excel(excel_table_path, sheet_name="valid_list")
tables = tables.loc[tables["VALID"].isin(["O", "o"])]
del tables["VALID"]
tables

In [ ]:
db_name = ""
schema_name = ""
excel_define_path = "data/input/defines/define_latest.xlsx"

for table_name in tables.loc[
    tables["DB"].str.contains(db_name) & tables["SCHEMA"].str.contains(schema_name)
]["TABLE"]:
    df = pd.read_excel(
        excel_define_path,
        sheet_name=table_name,
        skiprows=11,
        skipfooter=5,
        usecols="L:T",
    )
    df = df.loc[df["SEQ.1"].notnull()]
    run(df, table_name)

In [ ]:
from datetime import datetime, timedelta

start_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d 00:00:00")
end_date = datetime.now().strftime("%Y-%m-%d 00:00:00")
start_date, end_date